In [ ]:
import numpy as np

In [ ]:
def fit(X_train,Y_train):
    all_classes=set(Y_train)
    result={}
    result["total_data"]=len(X_train)
    for current_class in all_classes:
        result[current_class]={}
        current_class_rows=X_train[Y_train==current_class]
        result[current_class]["total_data"]=len(current_class_rows)
        num_features=X_train.shape[1]
        for i in range(1,num_features+1):
            result[current_class][i]={}
            all_labels=set(X_train[:,i-1]) # check
            for label in all_labels:
                result[current_class][i][label]=(current_class_rows[:,i-1]==label).sum()
    return result